In [8]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_data_40k.csv')
df = df[:1000]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_demo.csv',index=False)


In [48]:
df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_data_5k.csv')
df = df[:125]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_demo.csv',index=False)

In [15]:
import yaml
from easydict import EasyDict

with open('/hpc2hdd/home/yli106/smiles2mol/config/qm9_default.yml', 'r') as file:
    config = yaml.safe_load(file)

config = EasyDict(config)

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import argparse
from easydict import EasyDict
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("/hpc2hdd/home/yli106/smiles_3d/llama2_13b", trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token # Use the EOS token to pad shorter sequences
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

max_len = 0
for i in tqdm(range(len(df))):
    input_text = df['mol_block'][i]
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    if len(input_ids[0]) > max_len:
        max_len = len(input_ids[0])
max_len   

100%|██████████| 24068/24068 [01:15<00:00, 317.69it/s]


1739

In [6]:
smiles = df['canonicalize_smiles'][10]
system_prompt = 'Below is a SMILES of a molecule, generate its 3D structure. The molecule has 15 atoms and 15 bonds.'
inst = '<s>[INST] <<SYS>>\n' + system_prompt + '\n<</SYS>>\n\n' + smiles + ' [/INST] '+' </s>'
input_ids = tokenizer.encode(inst, return_tensors='pt')
len(input_ids[0])

105

In [1]:
import pickle

with open('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/test_data_200.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [1]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/test_data_200.csv')

In [3]:
df

,smiles,canonicalize_smiles,num_atom,num_bond,mol_block,smiles_index,conformers_index
0,C#CC(=O)[C@H](O)CCC,[H]C#CC(=O)[C@]([H])(O[H])C([H])([H])C([H])([H...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,0,0
1,C#CC(=O)[C@H](O)CCC,[H]C#CC(=O)[C@]([H])(O[H])C([H])([H])C([H])([H...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,0,1
2,C#CC(=O)[C@H](O)CCC,[H]C#CC(=O)[C@]([H])(O[H])C([H])([H])C([H])([H...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,0,2
3,C#CC(=O)[C@H](O)CCC,[H]C#CC(=O)[C@]([H])(O[H])C([H])([H])C([H])([H...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,0,3
4,C#CC(=O)[C@H](O)CCC,[H]C#CC(=O)[C@]([H])(O[H])C([H])([H])C([H])([H...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,0,4
...,...,...,...,...,...,...,...
24063,CO[C@H](C=O)CC(C)=O,[H]C(=O)[C@@]([H])(OC([H])([H])[H])C([H])([H])...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,199,52
24064,CO[C@H](C=O)CC(C)=O,[H]C(=O)[C@@]([H])(OC([H])([H])[H])C([H])([H])...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,199,53
24065,CO[C@H](C=O)CC(C)=O,[H]C(=O)[C@@]([H])(OC([H])([H])[H])C([H])([H])...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,199,54
24066,CO[C@H](C=O)CC(C)=O,[H]C(=O)[C@@]([H])(OC([H])([H])[H])C([H])([H])...,19,18,\n RDKit 3D\n\n 19 18 0 0 0 0...,199,55


In [1]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def preprocess(text):
    encoding = tokenizer.encode_plus(text, truncation=True, padding=True, return_tensors='pt')
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    return input_ids, attention_mask


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/hpc2hdd/home/yli106/smiles_3d/gpt2")


In [12]:
import pandas as pd
from datasets import Dataset 
df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_demo.csv')
train_dataset = Dataset.from_pandas(df[['text']])
test_dataset = train_dataset
train_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [7]:
from concurrent.futures import ThreadPoolExecutor

In [8]:
import torch

gpu_count = torch.cuda.device_count()
print("系统中的GPU个数为：", gpu_count)

系统中的GPU个数为： 0


In [10]:
texts_to_generate = [
    "Text 1", "Text 2", "Text 3", "Text 4",
    "Text 5", "Text 6", ["Text 7", "Text 8"],
    "Text 9", ["Text 10"]
]

# 计算需要分配到每个 GPU 的文本数量
num_gpus = 4
texts_per_gpu_floor = len(texts_to_generate) // num_gpus
texts_per_gpu_remainder = len(texts_to_generate) % num_gpus

# 创建一个列表，包含每个 GPU 分配的文本数量
texts_per_gpu_list = [texts_per_gpu_floor + 1 if i < texts_per_gpu_remainder else texts_per_gpu_floor for i in range(num_gpus)]

In [11]:
start = 0
sublists = []
for i in range(num_gpus):
    end = start + texts_per_gpu_list[i]
    sublist = texts_to_generate[start:end]
    sublists.append(sublist)
    start = end

In [12]:
sublists

[['Text 1', 'Text 2', 'Text 3'],
 ['Text 4', 'Text 5'],
 ['Text 6', ['Text 7', 'Text 8']],
 ['Text 9', ['Text 10']]]

In [13]:
i=1
def ke(j):
    print(j+i)

ke(3)

4
